# 空間を扱うプログラミング

## §2 カメラ

* 計算機上で 3D 空間を扱った後、スクリーン上に結果を表示する際には、3D 空間 → 2D 空間への座標変換 (投影変換) が発生する
* この座標変換は2ステップに分けられる。すなわち
  1. ワールド座標系 → カメラ座標系
  2. カメラ座標系 → 画像座標系
* 1番目のワールド座標系からカメラ座標系への変換では、ワールド上に置かれたカメラの座標・姿勢をもとに、カメラから 3D 空間がどのように見えるか計算する
  * この計算では、カメラの「外部パラメータ」を使用する
* 2番目のカメラ座標系から画像座標系への変換では、カメラから見た 3D 空間を 2D 画像上のピクセルへと落とし込む
  * この計算では、カメラの「内部パラメータ」を使用する

### §2-1 カメラ座標系と画像座標系

* カメラ座標系とは、ワールド上のカメラオブジェクトを考えたときの、カメラオブジェクトのローカル座標系でもある
* Open3D では、右手系でカメラオブジェクトの視線方向を +Z、カメラの右方向を +X、カメラの下方向を +Y と定義する

<img alt="Camera coordinate system in Open3D" src="./notebook_assets/images/1_example_transformation.png" style="width:600px" />

* 画像座標系とは、ピクセルの集合として表現されるスクリーン上でのオブジェクトの位置を表す座標系である
* Open3D では、スクリーンの左上の隅を原点とし、右方向を +X、下方向を +Y と定義する


<img alt="Screen coordinate system in Open3D" src="./notebook_assets/images/2_screen_coordinate_system.png" style="width:600px;" />


### §2-2 カメラモデル

* カメラを特徴づけるパラメータとして重要なのは下記の2つ
  * 外部パラメータ (extrinsic parameters)
    * ワールド座標系からカメラ座標系への変換を定義する
    * ここでは行列 $\mathbf{E}$ と表す
  * 内部パラメータ (intrinsic parameters)
    * カメラ座標系から画像座標系への変換を定義する
    * ここでは行列 $\mathbf{K}$ と表す

#### カメラ外部パラメータ

* 外部パラメータは、カメラ座標軸に対するワールド座標原点の位置と姿勢を表す transform として表現できる
  * カメラ座標系での、ワールド座標軸の並進ベクトルを $\mathbf{t}'$、 回転行列を $\mathbf{R}'$ (要素を $r_{ij}'$ とする) とすると下記のように書ける

$$
\mathbf{E} = \begin{bmatrix}
r'_{11} & r'_{12} & r'_{13} & t_x' \\
r'_{21} & r'_{22} & r'_{23} & t_y' \\
r'_{31} & r'_{32} & r'_{33} & t_z' \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

* ワールド座標系でのカメラオブジェクトの位置姿勢から計算する場合、ワールド座標系でのカメラの並進ベクトルを $\mathbf{t}$、 回転行列を $\mathbf{R}$ とすると、下記のように計算できる

$$
\mathbf{R}' = \mathbf{R}^T (転置行列) \\
\mathbf{t}' = - \mathbf{R}^T \mathbf{t}
$$

* カメラ外部パラメータを用いると、ワールド上の任意の点 $\mathbf{p}$ のカメラ座標系での座標 $\mathbf{p}_C$ は下記のように書ける

$$
\mathbf{p}_C = \mathbf{E} \mathbf{p} \\
ただし \\
\mathbf{p} = (x, y, z, 1) \\
\mathbf{p}_C = (x_C, y_C, z_C, 1)
$$

#### カメラ内部パラメータ

* 内部パラメータは、カメラ座標系で表される 3D 空間の点から、画像座標系で表される 2D スクリーン上の点への変換を特徴づけるパラメータである
  * 内部パラメータは、ワールド上でのカメラの位置・姿勢には依存しないパラメータである
* 内部パラメータはカメラのレンズのモデルに応じて構成要素が異なるが、ここでは最も簡単で基本的な `ピンホールカメラ` を扱う
* ピンホールカメラの内部パラメータは、レンズの焦点距離 (focal length) と主点 (principal point) で定義される
  * 焦点距離はレンズ中心と焦点間の距離を表し、単位は pixel とする
    * (光の波長による違いを無視すると) レンズの焦点距離はただ一つの値を取るが、カメラのイメージセンサーの仕様 (画素の縦横比が1ではないなど) によっては、pixel 換算の焦点距離がスクリーンの X方向・Y方向で違う場合がある
  * 主点は画像座標における、レンズの軸とスクリーンの交点の座標である
    * 簡易的にスクリーンの幅・高さの半分で近似することが多い
* 焦点距離をスクリーンの X方向・Y方向でそれぞれ $f_X$, $f_Y$、主点を $(c_X, c_Y)$ としたとき、カメラ座標系から画像座標系への変換行列 (K 行列) $\mathbf{K}$ は下記のように書ける

$$
\mathbf{K} = \begin{bmatrix}
f_X &   0 & c_X & 0 \\
  0 & f_Y & c_Y & 0 \\
  0 &   0 &   1 & 0 \\
\end{bmatrix}
$$

* K 行列を用いると、カメラ座標系上の任意の点 $\mathbf{p}_C$ の画像座標系での座標 $\mathbf{p}_S$ は下記のように書ける

$$
\mathbf{p}_S = \mathbf{K} \mathbf{p}_C \\
ただし \\
\mathbf{p}_C = (x_C, y_C, z_C, 1) \\
\mathbf{p}_S = (x_S, y_S, 1)
$$

* 実際のカメラレンズでは、スクリーン中心から離れるに従って像が歪むことを考慮することがある
* また、パノラマカメラや魚眼カメラのように広範囲を写すカメラレンズでは、ピンホールカメラモデルよりも複雑なカメラモデルを使用することがある

#### 投影変換

* カメラ外部パラメータ・内部パラメータで特徴づけられる K 行列により、ワールド座標上の任意の点 $\mathbf{p}$ の画像座標系での座標 $\mathbf{p}_S$ は下記のように書ける

$$
\mathbf{p}_S = \mathbf{K} \mathbf{p}_C = \mathbf{K} \mathbf{E} \mathbf{p}
$$

* 上記において $\mathbf{K} \mathbf{E}$ で表される行列を投影行列という

In [ ]:
# 必要なライブラリをインポート
from __future__ import annotations

import numpy as np
import open3d as o3d
import sympy as sp
from IPython.display import Math, display
from scipy.spatial.transform import Rotation

from util_lib.camera import SimplePinholeCamera
from util_lib.transformable_object import TransformableObject
from util_lib.types import EulerOrder, PinholeCameraParameters, Transform
from util_lib.visualization import draw_geometries
from util_lib.world import create_coordinate_objects

# ワールド・カメラ座標系の作成と読み込み
world_coordinate: o3d.geometry.Geometry = create_coordinate_objects()
original_camera = TransformableObject.load_model("./data/camera.gltf", enable_post_processing=True)


# 行列表示用の関数
def print_matrix(mat: np.ndarray | Transform) -> None:
    if isinstance(mat, Transform):
        mat_sym = sp.Matrix(mat.get_matrix())
    else:
        mat_sym = sp.Matrix(mat)
    mat_sym = mat_sym.applyfunc(lambda x: sp.Symbol(f"{x:.3f}"))
    display(Math(sp.latex(mat_sym)))


初期配置されたカメラの視点を再現

* Open3D のカメラモデルはピンホールカメラモデル
* スクリーンサイズは FullHD (幅 1920 pixel、高さ 1080 px) とする
* カメラ内部パラメータとして下記を設定する
  * 主点はスクリーンの中央とする
  * 焦点距離は 600 px とする (小さくすると一度に写る範囲が広くなる)

下記のように、カメラオブジェクトにくっついているローカル座標軸と、ワールド座標軸の一部が見えるはず
(見えなければズームアウトしてみてください)

<img alt="View of initial camera object" src="./notebook_assets/images/2_initial_camera_view.png" style="width:600px;" />

In [ ]:
# ピンホールカメラ内部パラメータ
image_size = (1920, 1080) # pixels (FullHD)
focal_length = 600 # pixels
principal_point = (image_size[0] / 2 - 0.5, image_size[1] / 2 - 0.5) # pixels
camera_parameters = PinholeCameraParameters(focal_length, principal_point, image_size)

# 初期カメラ姿勢の設定
# ワールドに初期配置されているカメラオブジェクトのローカル座標系と、カメラ座標系の位置と向きを合わせておく
initial_rotate = Rotation.from_euler(EulerOrder.xyz, [-90, 180, 0], degrees=True).as_matrix()
initial_pose = Transform.from_rotate_and_translate(initial_rotate, [0, 0, 0])
view_camera = SimplePinholeCamera(camera_parameters, initial_pose)

In [ ]:
print("K 行列 (k_11 ~ k_33 部分のみ)")
print_matrix(view_camera.get_intrinsic_matrix())
print("外部パラメータ")
print_matrix(view_camera.get_extrinsic_matrix())
draw_geometries(world_coordinate, original_camera, camera=view_camera)

### 任意の場所のカメラ視点を再現

* ここではワールド座標 (x, y, z) = (1, 1, 1)、ワールド座標軸に対する Euler 角 (y → x → z): (180°, -135°, 0°) の姿勢を持つカメラ視点を再現してみる

<img alt="Example camera pose" src="./notebook_assets/images/2_example_camera_pose.png" style="width:600px;" />

<img alt="Example camera view" src="./notebook_assets/images/2_example_camera_view.png" style="width:600px;" />

In [ ]:
# カメラオブジェクトの移動と回転
camera_obj = original_camera.copy()
camera_obj.translate([1, 1, 1])
camera_obj.rotate_by_euler(EulerOrder.yxz, [180, -135, 0])

# ワールド座標系でカメラを移動・回転 (カメラオブジェクトから transform を取得)
view = view_camera.copy()
transform = camera_obj.get_transform()
view.transform(transform)

print("ワールド座標系でのカメラオブジェクトの位置・姿勢")
print_matrix(transform)

print("カメラ外部パラメータ")
print_matrix(view.get_extrinsic_matrix())

print("カメラ外部パラメータ (Euler 角と並進ベクトル")
print_matrix(view.get_extrinsic_parameters(EulerOrder.xyz)[0])
print_matrix(view.get_extrinsic_parameters(EulerOrder.xyz)[1])

draw_geometries(world_coordinate, original_camera, camera_obj, camera=view)

### 外部パラメータの解釈

* 外部パラメータとは、カメラ座標系に対するワールド座標系の座標原点の位置・座標軸の姿勢を表す transformation
* 上の例のカメラのカメラ座標軸を基準に考えたとき
  * ワールド座標原点は (X, Y, Z) = (-1, 0, $\sqrt{2}$) にあるように見える
    * (大きい座標軸の矢印の根本は、便宜上ワールド座標原点から少しずれていることに注意)
  * ワールド座標軸の姿勢は、カメラ座標系をカメラ座標系での Euler 角 (X → Y → Z): (-135°, 0, 0) で回転させたように見える

<img alt="Example extrinsic parameter" src="./notebook_assets/images/2_camera_extrinsic_1.png" style="width:600px;" />
<img alt="Example extrinsic parameter" src="./notebook_assets/images/2_camera_extrinsic_2.png" style="width:600px;" />